In [1]:
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')


In [2]:
customers['SignupDate'] = pd.to_datetime(customers['SignupDate'])
transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'])

max_transaction_date = transactions['TransactionDate'].max()
max_transaction_date

Timestamp('2024-12-28 11:00:00')

In [3]:
customers['tenure_days'] = (max_transaction_date - customers['SignupDate']).dt.days

merged = pd.merge(transactions, products[['ProductID', 'Category']], on='ProductID', how='left')
merged.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price,Category
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Electronics
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Electronics
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Electronics
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Electronics
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Electronics


In [4]:
transaction_agg = merged.groupby('CustomerID').agg(
    total_transactions=('TransactionID', 'count'),
    total_quantity=('Quantity', 'sum'),
    total_spent=('TotalValue', 'sum'),
    avg_price=('Price', 'mean'),
    unique_products=('ProductID', 'nunique')
).reset_index()
transaction_agg


,CustomerID,total_transactions,total_quantity,total_spent,avg_price,unique_products
0,C0001,5,12,3354.52,278.334000,5
1,C0002,4,10,1862.74,208.920000,4
2,C0003,4,14,2725.38,195.707500,4
3,C0004,8,23,5354.88,240.636250,8
4,C0005,3,7,2034.24,291.603333,3
...,...,...,...,...,...,...
194,C0196,4,12,4982.88,416.992500,3
195,C0197,3,9,1928.65,227.056667,3
196,C0198,2,3,931.83,239.705000,2
197,C0199,4,9,1979.28,250.610000,4


In [5]:
transaction_agg['avg_transaction_value'] = transaction_agg['total_spent'] / transaction_agg['total_transactions']
transaction_agg['avg_quantity_per_transaction'] = transaction_agg['total_quantity'] / transaction_agg['total_transactions']

transaction_agg

,CustomerID,total_transactions,total_quantity,total_spent,avg_price,unique_products,avg_transaction_value,avg_quantity_per_transaction
0,C0001,5,12,3354.52,278.334000,5,670.904000,2.400000
1,C0002,4,10,1862.74,208.920000,4,465.685000,2.500000
2,C0003,4,14,2725.38,195.707500,4,681.345000,3.500000
3,C0004,8,23,5354.88,240.636250,8,669.360000,2.875000
4,C0005,3,7,2034.24,291.603333,3,678.080000,2.333333
...,...,...,...,...,...,...,...,...
194,C0196,4,12,4982.88,416.992500,3,1245.720000,3.000000
195,C0197,3,9,1928.65,227.056667,3,642.883333,3.000000
196,C0198,2,3,931.83,239.705000,2,465.915000,1.500000
197,C0199,4,9,1979.28,250.610000,4,494.820000,2.250000


In [6]:
category_counts = pd.crosstab(merged['CustomerID'], merged['Category'])

category_counts

Category,Books,Clothing,Electronics,Home Decor
CustomerID,,,,
C0001,1,0,3,1
C0002,0,2,0,2
C0003,0,1,1,2
C0004,3,0,2,3
C0005,0,0,2,1
...,...,...,...,...
C0196,1,1,0,2
C0197,0,0,2,1
C0198,0,1,1,0


In [7]:
total_trans = transaction_agg.set_index('CustomerID')['total_transactions']
category_proportions = category_counts.div(total_trans, axis=0).fillna(0).reset_index()
category_proportions

Category,CustomerID,Books,Clothing,Electronics,Home Decor
0,C0001,0.200,0.00,0.600000,0.200000
1,C0002,0.000,0.50,0.000000,0.500000
2,C0003,0.000,0.25,0.250000,0.500000
3,C0004,0.375,0.00,0.250000,0.375000
4,C0005,0.000,0.00,0.666667,0.333333
...,...,...,...,...,...
194,C0196,0.250,0.25,0.000000,0.500000
195,C0197,0.000,0.00,0.666667,0.333333
196,C0198,0.000,0.50,0.500000,0.000000
197,C0199,0.000,0.00,0.500000,0.500000


In [8]:
features = pd.merge(customers, transaction_agg, on='CustomerID', how='left')
features = pd.merge(features, category_proportions, on='CustomerID', how='left')
features.fillna(0, inplace=True)

features

,CustomerID,CustomerName,Region,SignupDate,tenure_days,total_transactions,total_quantity,total_spent,avg_price,unique_products,avg_transaction_value,avg_quantity_per_transaction,Books,Clothing,Electronics,Home Decor
0,C0001,Lawrence Carroll,South America,2022-07-10,902,5.0,12.0,3354.52,278.334000,5.0,670.904000,2.400000,0.200,0.00,0.600000,0.200000
1,C0002,Elizabeth Lutz,Asia,2022-02-13,1049,4.0,10.0,1862.74,208.920000,4.0,465.685000,2.500000,0.000,0.50,0.000000,0.500000
2,C0003,Michael Rivera,South America,2024-03-07,296,4.0,14.0,2725.38,195.707500,4.0,681.345000,3.500000,0.000,0.25,0.250000,0.500000
3,C0004,Kathleen Rodriguez,South America,2022-10-09,811,8.0,23.0,5354.88,240.636250,8.0,669.360000,2.875000,0.375,0.00,0.250000,0.375000
4,C0005,Laura Weber,Asia,2022-08-15,866,3.0,7.0,2034.24,291.603333,3.0,678.080000,2.333333,0.000,0.00,0.666667,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,C0196,Laura Watts,Europe,2022-06-07,935,4.0,12.0,4982.88,416.992500,3.0,1245.720000,3.000000,0.250,0.25,0.000000,0.500000
196,C0197,Christina Harvey,Europe,2023-03-21,648,3.0,9.0,1928.65,227.056667,3.0,642.883333,3.000000,0.000,0.00,0.666667,0.333333
197,C0198,Rebecca Ray,Europe,2022-02-27,1035,2.0,3.0,931.83,239.705000,2.0,465.915000,1.500000,0.000,0.50,0.500000,0.000000
198,C0199,Andrea Jenkins,Europe,2022-12-03,756,4.0,9.0,1979.28,250.610000,4.0,494.820000,2.250000,0.000,0.00,0.500000,0.500000


In [9]:
encoder = OneHotEncoder()
region_encoded = encoder.fit_transform(features[['Region']]).toarray()
region_encoded_df = pd.DataFrame(region_encoded, columns=encoder.get_feature_names_out(['Region']))
features = pd.concat([features.drop('Region', axis=1), region_encoded_df], axis=1)

features

,CustomerID,CustomerName,SignupDate,tenure_days,total_transactions,total_quantity,total_spent,avg_price,unique_products,avg_transaction_value,avg_quantity_per_transaction,Books,Clothing,Electronics,Home Decor,Region_Asia,Region_Europe,Region_North America,Region_South America
0,C0001,Lawrence Carroll,2022-07-10,902,5.0,12.0,3354.52,278.334000,5.0,670.904000,2.400000,0.200,0.00,0.600000,0.200000,0.0,0.0,0.0,1.0
1,C0002,Elizabeth Lutz,2022-02-13,1049,4.0,10.0,1862.74,208.920000,4.0,465.685000,2.500000,0.000,0.50,0.000000,0.500000,1.0,0.0,0.0,0.0
2,C0003,Michael Rivera,2024-03-07,296,4.0,14.0,2725.38,195.707500,4.0,681.345000,3.500000,0.000,0.25,0.250000,0.500000,0.0,0.0,0.0,1.0
3,C0004,Kathleen Rodriguez,2022-10-09,811,8.0,23.0,5354.88,240.636250,8.0,669.360000,2.875000,0.375,0.00,0.250000,0.375000,0.0,0.0,0.0,1.0
4,C0005,Laura Weber,2022-08-15,866,3.0,7.0,2034.24,291.603333,3.0,678.080000,2.333333,0.000,0.00,0.666667,0.333333,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,C0196,Laura Watts,2022-06-07,935,4.0,12.0,4982.88,416.992500,3.0,1245.720000,3.000000,0.250,0.25,0.000000,0.500000,0.0,1.0,0.0,0.0
196,C0197,Christina Harvey,2023-03-21,648,3.0,9.0,1928.65,227.056667,3.0,642.883333,3.000000,0.000,0.00,0.666667,0.333333,0.0,1.0,0.0,0.0
197,C0198,Rebecca Ray,2022-02-27,1035,2.0,3.0,931.83,239.705000,2.0,465.915000,1.500000,0.000,0.50,0.500000,0.000000,0.0,1.0,0.0,0.0
198,C0199,Andrea Jenkins,2022-12-03,756,4.0,9.0,1979.28,250.610000,4.0,494.820000,2.250000,0.000,0.00,0.500000,0.500000,0.0,1.0,0.0,0.0


In [10]:
feature_columns = [
    'tenure_days', 'total_transactions', 'total_quantity', 'total_spent',
    'avg_transaction_value', 'avg_quantity_per_transaction', 'unique_products', 'avg_price'
] + list(category_proportions.columns[1:]) + list(region_encoded_df.columns)

X = features[feature_columns]

feature_columns

['tenure_days',
 'total_transactions',
 'total_quantity',
 'total_spent',
 'avg_transaction_value',
 'avg_quantity_per_transaction',
 'unique_products',
 'avg_price',
 'Books',
 'Clothing',
 'Electronics',
 'Home Decor',
 'Region_Asia',
 'Region_Europe',
 'Region_North America',
 'Region_South America']

In [11]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [12]:
similarity_matrix = cosine_similarity(X_scaled)

In [13]:
target_customer_ids = [f'C00{i:02d}' for i in range(1, 21)]
results = []

for target_id in target_customer_ids:
    target_index = features[features['CustomerID'] == target_id].index[0]
    similarities = similarity_matrix[target_index]
    customer_similarities = []
    for idx, score in enumerate(similarities):
        customer_id = features.iloc[idx]['CustomerID']
        if customer_id == target_id:
            continue
        customer_similarities.append((customer_id, score))
    customer_similarities.sort(key=lambda x: x[1], reverse=True)
    top_3 = customer_similarities[:3]
    results.append((target_id, top_3))

In [14]:
rows = []
for target_id, top_3 in results:
    row = [target_id]
    for cust_id, score in top_3:
        row.extend([cust_id, round(score, 4)])
    rows.append(row)

columns = ['CustomerID', 'Lookalike1', 'Score1', 'Lookalike2', 'Score2', 'Lookalike3', 'Score3']
lookalike_df = pd.DataFrame(rows, columns=columns)

lookalike_df.to_csv('Atishay_Jain_Lookalike.csv', index=False)

In [15]:
lookalike_df

,CustomerID,Lookalike1,Score1,Lookalike2,Score2,Lookalike3,Score3
0,C0001,C0112,0.8035,C0120,0.7990,C0192,0.7534
1,C0002,C0106,0.9065,C0159,0.8954,C0134,0.8842
2,C0003,C0129,0.8186,C0151,0.7835,C0031,0.7773
3,C0004,C0113,0.9527,C0104,0.8594,C0102,0.8323
4,C0005,C0007,0.9469,C0140,0.8789,C0186,0.7892
5,C0006,C0187,0.7975,C0171,0.7580,C0168,0.6728
6,C0007,C0005,0.9469,C0140,0.8739,C0186,0.7286
7,C0008,C0098,0.8606,C0194,0.8579,C0024,0.8122
8,C0009,C0198,0.8782,C0058,0.7503,C0061,0.7498
9,C0010,C0062,0.8237,C0061,0.7078,C0111,0.7058
